In [8]:
import retrieval.cache as cache
import signals.ema as ema

import pandas as pd
import pandas_ta as ta
from tqdm import tqdm

### Make sure we have some data to work with

In [9]:
ticker = 'BTC-USD'
start = '2023-01-01'
end = '2023-12-31'
interval = '1d'

if not cache.exist_sql_db(ticker, interval):
    print(f'''Missing {ticker} {interval} in SQL DB... adding now.''')
    cache.cache_ticker(ticker, interval, start, end)

df = cache.load_ticker(ticker, interval, start, end)

#df

### Add TA Columns

In [10]:
df['EMA Slow'] = ta.ema(df['Close'], length=50)
df['EMA Fast'] = ta.ema(df['Close'], length=30)
df['RSI'] = ta.rsi(df['Close'], length=10)

# Average True Range (or Volatility) - help to define stop loss / take profit distance
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=7)

# Bollinger bands
df=df.join(ta.bbands(df['Close'], length=15, std=1.5))

#df


### EMA signal function

In [ ]:
tqdm.pandas()

df.reset_index(inplace=True)

# row.name is the index of the row -> 'Date' column
df['EMA Signal'] = df.progress_apply(
  lambda row: ema.two_above_or_below(df, row.name, 7) if row.name >= 20 else 0, 
  axis=1
)

#df